In [11]:
#### Libraries ####
## Python Lib ##
import os

# Extended Libs #
import numpy as np
import pandas as pd
import json

# Plotting Libs #
import folium
import seaborn as sns
import matplotlib.pyplot as plt

# Data processing & matching #
from scipy.spatial.distance import cdist
from scipy.interpolate import CubicSpline, interp1d
import geopandas as gpd
from shapely.geometry import Point, box
from roaring_landmask import RoaringLandmask

# Choose file #
import tkinter as tk

# Own functions #
from functions import DataProcessor

In [2]:
### PLOT OG SPLINE ###
def plot_splines_ais_sar_on_map(splines=None, ais_mmsi=None, sar_data=None, norsat_data=None, ais_sar_pol = None):
    # Create a base map centered around the mean of all AIS points
    map_center = [
        sum(value['y'].mean() for _, value in splines.items()) / len(splines),
        sum(value['x'].mean() for _, value in splines.items()) / len(splines),
    ]

    m = folium.Map(location=map_center, zoom_start=6)

    # Plot splines
    #if splines is not None:
     #   for key, value in splines.items():
      #      spline_coords = list(zip(value['y'], value['x']))
       #     folium.PolyLine(spline_coords, color='blue', weight=2.5, opacity=1).add_to(m)

            
    # Extract corresponding AIS data
    if ais_mmsi is not None and not ais_mmsi.empty:
        tes = ais_mmsi.get_group((key,))  # Pass key as a tuple
        for _, row in tes.iterrows():
                folium.CircleMarker(
                    location=[row['latitude'], row['longitude']],
                    radius=1.5,
                    color='red',
                    fill=True,
                    fill_opacity=0.7,
                    popup=f"AIS MMSI: {row['mmsi']}"
                ).add_to(m)

    if ais_sar_pol is not None: # Add markers for each AIS/SAR point
        for mmsi, coords in ais_sar_pol.items():
            folium.CircleMarker(
				location=[coords['y'], coords['x']],  # Latitude first, then Longitude
                radius=1.5,
                color='blue',
                fill=True,
                fill_opacity=0.7,
				popup=f"MMSI: {mmsi}",
				#icon=folium.Icon(color='blue', icon='info-sign')
			).add_to(m)
			

    # Plot SAR points if the DataFrame is not None and not empty
    if sar_data is not None and not sar_data.empty:
        for _, row in sar_data.iterrows():
            folium.CircleMarker(
                location=[row['latitude'], row['longitude']],
                radius=2,
                color='green',
                fill=True,
                fill_opacity=0.7,
                popup=f"SAR Object ID: {row['Object_ID']}"
            ).add_to(m)

    # Plot Norsat points if the DataFrame is not None and not empty
    if norsat_data is not None and not norsat_data.empty:
        for idx, row in norsat_data.iterrows():
            folium.CircleMarker(
                location=[row['latitude'], row['longitude']],
                radius=5,
                color='black',
                fill=True,
                fill_opacity=0.7,
                popup=f"idx: {idx}"
            ).add_to(m)

    return m

In [3]:
# Define paths
base_path = "C:\\Users\\abelt\\OneDrive\\Desktop\\Kandidat\\"

# File names
ais_files = {
    '02-11-2022': 'ais\\ais_110215.csv',
    '03-11-2022': 'ais\\ais_110315.csv',
    '05-11-2022': 'ais\\ais_1105.csv'
}

sar_files = {
    '02-11-2022': 'sar\\Sentinel_1_detection_20221102T1519.json',
    '03-11-2022': 'sar\\Sentinel_1_detection_20221103T154515.json',
    '05-11-2022': 'sar\\Sentinel_1_detection_20221105T162459.json'
}

norsat_files = {
    '02-11-2022': 'norsat\\Norsat3-N1-JSON-Message-DK-2022-11-02T151459Z.json',
    '03-11-2022': 'norsat\\Norsat3-N1-JSON-Message-DK-2022-11-03T152759Z.json',
    '05-11-2022': 'norsat\\Norsat3-N1-JSON-Message-DK-2022-11-05T155259Z.json'
}

In [4]:
# Define date and time filter
date_key = '03-11-2022'
delta_time = pd.Timedelta(days = 0, hours = 5, minutes = 0)

In [5]:
# Loading and filtering data #
# Initialize the DataProcessor class #
data_processor = DataProcessor(base_path)

# Load AIS, SAR, and Norsat data
data_processor.load_ais_data(ais_files)
data_processor.load_sar_data(sar_files)
data_processor.load_norsat_data(norsat_files)

# Expand SAR data for the selected date
expanded_sar_df = data_processor.expand_objects_for_date(date_key)
# Filter SAR data (only class 0 "ship"-like)
filtered_sar_df = expanded_sar_df #.loc[expanded_sar_df['class'] == 0]
# Applying landmask and appending if ship is on land
filtered_sar_df = data_processor.filter_sar_landmask(filtered_sar_df)
# Filter AIS data based on SAR timestamps
filtered_ais_df = data_processor.filter_ais_data(date_key, delta_time)

# Norsat data doesn't need filtering for now
filtered_norsat_df = data_processor.dfs_norsat[date_key]

# Store filtered data
filter_dfs = {'AIS': filtered_ais_df, 'SAR': filtered_sar_df, 'Norsat': filtered_norsat_df}

# Convert columns to appropriate data types
data_processor.convert_data_types(expanded_sar_df, ['Start'], ['latitude', 'longitude'])
data_processor.convert_data_types(filter_dfs['AIS'], ['TimeStamp'], ['latitude', 'longitude'])
data_processor.convert_data_types(filter_dfs['SAR'], ['Start'], ['latitude', 'longitude'])


# Clean data by removing rows with NaN in latitude and longitude
expanded_sar_df = data_processor.clean_data(expanded_sar_df, ['latitude', 'longitude'])
filter_dfs['AIS'] = data_processor.clean_data(filter_dfs['AIS'], ['latitude', 'longitude'])

# Display data structure
data_processor.display_data_structure()

AIS:
dict_keys(['02-11-2022', '03-11-2022', '05-11-2022'])
Columns: Index(['time', 'callsign', 'cog', 'destination', 'draught', 'eta', 'imo',
       'latitude', 'length', 'longitude', 'mmsi', 'nav_status', 'name', 'sog',
       'shiptype', 'width', 'source', 'TimeStamp'],
      dtype='object')
SAR:
dict_keys(['02-11-2022', '03-11-2022', '05-11-2022'])
Columns: Index(['ProductType', 'Polarization', 'Swath', 'Start', 'End', 'Name',
       'Satellite', 'Shape', 'Objects', 'source'],
      dtype='object')
Norsat:
dict_keys(['02-11-2022', '03-11-2022', '05-11-2022'])
Columns: Index(['TimeStamp', 'CollectionInformation', 'NRDEmitterPosition',
       'NumberCandidates', 'CandidateList', 'source', 'latitude', 'longitude'],
      dtype='object')


In [6]:
# Grouping mmsi points and interpolations for temporal matching #
ais_mmsi = filter_dfs['AIS'].groupby(by=['mmsi'])

mmsi_numbers = list(ais_mmsi.indices.keys())

splines = {}
not_enough = []  # List to store MMSI numbers with insufficient data points

for mmsi in mmsi_numbers:
    tes = ais_mmsi.get_group((mmsi,))

    # Check for NaN values and drop them
    if tes['TimeStamp'].isnull().any() or tes['longitude'].isnull().any() or tes['latitude'].isnull().any():
        continue  # Skip this group if there are NaN values

    # Ensure there are at least two points for interpolation
    if len(tes) < 2:
        not_enough.append(mmsi)  # Append MMSI to the list
        #print(f"Not enough data points for MMSI {mmsi}. Skipping interpolation.")
        continue

    # Create time series for interpolation
    times = pd.to_numeric(pd.date_range(start=tes['TimeStamp'].min(), end=tes['TimeStamp'].max(), periods=len(tes)*2))

    # Linear interpolation (works even for 2 points)
    int_linear = interp1d(pd.to_numeric(tes['TimeStamp']), np.c_[tes['longitude'], tes['latitude']], axis=0, fill_value="extrapolate", kind='linear')

    # Cubic spline interpolation, only if there are more than 3 points
    if len(tes) > 3:
        spl_cubic = CubicSpline(pd.to_numeric(tes['TimeStamp']), np.c_[tes['longitude'], tes['latitude']])
		
    # Store the results in the splines dictionary
    splines[mmsi] = {
        'cubic': spl_cubic,
        'linear': int_linear
    }

# After processing, you can check the not_enough list
print(f"MMSI numbers with insufficient data points: {len(not_enough)} out of {len(mmsi_numbers)}")¨
print(f"")

MMSI numbers with insufficient data points: 99 out of 2665


In [7]:
xy_interpolated = {}
# Create time series for interpolation
for mmsi in mmsi_numbers:
    tes = ais_mmsi.get_group((mmsi,))

    # Check for NaN values and drop them
    if tes['TimeStamp'].isnull().any() or tes['longitude'].isnull().any() or tes['latitude'].isnull().any():
        continue  # Skip this group if there are NaN values

    times = pd.to_numeric(pd.date_range(start=tes['TimeStamp'].min(), end=tes['TimeStamp'].max(), periods=len(tes)*2))

    if len(tes) > 1:
        x_linear, y_linear = splines[mmsi]['linear'](times).T
    else:
        continue
	
    x_cubic, y_cubic = splines[mmsi]['cubic'](times).T if len(tes) > 3 else (x_linear, y_linear)

    xy_interpolated[mmsi] = {'x' : x_cubic, 'y' : y_cubic}


In [8]:
## LAND FILTER ##
filter_df_sar_0 = filter_dfs['SAR'][filter_dfs['SAR']['on_land'] == 0]
filter_df_sar_1 = filter_dfs['SAR'][filter_dfs['SAR']['on_land'] == 1]
print(f'## Landmask filter: ##\nOn water: {len(filter_df_sar_0)}, on land: {len(filter_df_sar_1)} \nTotal number of SAR points: {len(filter_dfs['SAR'])}')

## Landmask filter: ##
On water: 6186, on land: 19031 
Total number of SAR points: 25217


In [9]:
ais_sar_pol = {}
for mmsi in mmsi_numbers:
	tes = ais_mmsi.get_group((mmsi,))

	# Convert 'Start' column to nanoseconds since the epoch (same as pd.to_numeric(pd.date_range(...)))
	time_convert = pd.to_numeric(pd.to_datetime(tes['time']))
		# Now you can compute the mean of the numeric values
	time_interpol = time_convert.mean()

	time = pd.to_numeric(time_interpol)

	if len(tes) > 1:
			x_linear, y_linear = splines[mmsi]['linear'](time)
	else:
			continue

	x_cubic, y_cubic = splines[mmsi]['cubic'](time) if len(tes) > 3 else (x_linear, y_linear)
		
	ais_sar_pol[mmsi] = {'x' : x_cubic, 'y' : y_cubic}


In [10]:
# Call the function
#m = plot_splines_ais_sar_on_map(splines = xy_interpolated, ais_mmsi = ais_mmsi, sar_data = filter_df_sar_0, norsat_data = filter_dfs['Norsat'])
m = plot_splines_ais_sar_on_map(splines = xy_interpolated, ais_mmsi = None, sar_data = filter_df_sar_0, norsat_data = None, ais_sar_pol = ais_sar_pol)
m.save(f'./images/splines_ais_sar_map_{date_key}_on_land_false.html')  # Save to an HTML file if needed

# Call the function
m = plot_splines_ais_sar_on_map(splines = xy_interpolated, ais_mmsi = None, sar_data = filter_df_sar_1, norsat_data = None, ais_sar_pol = ais_sar_pol)
m.save(f'./images/splines_ais_sar_map_{date_key}_on_land_true.html')  # Save to an HTML file if needed

Matching:
- SAR & AIS
- SAR & RF
- AIS & RF
- SAR, AIS, & RF
Create uncertainty chart or correlation value chart.